# Flow ML Course

# TP 6

L'objectif de ce TP est d'utiliser tous les outils vus dans les TP précédents pour entrainer et déposer un modèle sur la plateforme de Flow ML de GCP.

## Initialisation

In [1]:
! pip3 install --upgrade --quiet google-cloud-aiplatform[tensorboard] \
                                 tensorflow==2.15.1

In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

# Clear any logs from previous runs
!rm -rf ./logs/


In [28]:
your_name = ...

In [3]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import joblib
import os
import gcsfs

from google.cloud import aiplatform
from google.cloud import storage

# Import TensorFlow and the TensorBoard HParams plugin
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp


PROJECT_ID = "projet-ia-448520"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

aiplatform.init(project=PROJECT_ID, location=LOCATION)

BUCKET_URI = "gs://cours1bucket" 



client = storage.Client()
bucket = client.bucket(BUCKET_URI.split("//")[1])


2025-02-21 10:43:07.606554: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-21 10:43:07.606615: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-21 10:43:07.608980: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-21 10:43:07.622724: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Extraction des données

Depuis un bucket GCP, on utilisera gcsfs.

Utilisez l'exemple du fichier exams_train.csv pour extraire exams_test.csv

Le fichier de base est disponible ici : https://www.kaggle.com/datasets/whenamancodes/students-performance-in-exams?resource=download

In [ ]:


fs = gcsfs.GCSFileSystem(project=PROJECT_ID)
with fs.open(f"{BUCKET_URI.split('//')[1]}/data/exams_train.csv") as f:
    df_train = pd.read_csv(f)
df_train

In [ ]:

## Your Turn 


df_test = ...

## Explore & Transform your data

L'objectif est de prédire le "math score".

Analyser, explorer et transformer les données pour pouvoir prédire le score.

In [54]:
...

## Training

### Model Selection

Choisissez 3 modèles à entrainer pour répondre à la problématique du dataset

In [35]:
model_names = [...]

### Training Method


Utiliser cet espace pour créer les méthodes vous aidant à entrainer les modèles


Conseil :

 Enregistrez vos modèles à chacun de vos entrainements. Enregistrer dans l'espace de storage google via ce snippet :


'''

    blob = bucket.blob(MODEL_DIR)

    blob.upload_from_filename(model_file)
'''


Modifier MODEL_DIR pour créer un sous répertoire à votre nom où vous pourrez stocker vos informations.

### Hyperparamétrage

Définisser les range d'hyperparamètres ainsi que la métrique d'évaluation ici

In [190]:
...

### Train

Entrainer vos différents modèles avec les différentes combinaisons de paramètres.

In [ ]:
...

### Tensorboard

Poussez chaque modèle dans des des expériments Tensorboards

In [ ]:
# Set the display name for your tensorboard instance

for model_name in model_names:
    TENSORBOARD_NAME = f"supaero-vertex-course-1-6-tb-{PROJECT_ID}-{your_name}"  # @param {type:"string"}

    tensorboard = aiplatform.Tensorboard.create(
        display_name=TENSORBOARD_NAME, project=PROJECT_ID, location=LOCATION
    )
    TENSORBOARD_RESOURCE_NAME = tensorboard.gca_resource.name
    print("TensorBoard resource name:", TENSORBOARD_RESOURCE_NAME)

    EXPERIMENT_NAME = f"supaero-vertex-course-1-6-{PROJECT_ID}-{your_name}-{model_name.lower()}"  # @param {type:"string"}

    !tb-gcp-uploader --one_shot=True --tensorboard_resource_name=$TENSORBOARD_RESOURCE_NAME --logdir="logs/hparam_tuning_model_1/" --experiment_name=$EXPERIMENT_NAME

### Meilleurs modèles

Utiliser Tensorboard pour définir les meilleurs paramètres pour chaque modèle

In [ ]:
...

## Model Registry

Sauvegardez vos modèles dans le model Registry de ML Flow.

Utiliser ce lien pour définir les DEPLOY_IMAGE en fonction de votre type de modèle (CPU only) : https://cloud.google.com/vertex-ai/docs/training/pre-built-containers#scikit-learn

In [81]:
aiplatform.Model.upload(...)

## Versionning

Update the model registry to get the model in version v1

In [233]:
aiplatform.Model.list(filter=f'display_name="{...}"')

In [ ]:
...

## Model Lineage



## Endpoint

Créer un endpoint

In [ ]:

version_selected = ...


# Deploy the model to the endpoint
endpoint = aiplatform.Endpoint.create(
        display_name=...,
        project=PROJECT_ID,
        location=LOCATION,
    )


models = aiplatform.Model.list(
    filter=f'display_name="{...}"'
    )
print(models)


# Vous pouvez également utiliser l'interface web si vous rencontrez des difficultées
models[version_selected].deploy(
    deployed_model_display_name=...,
    machine_type='n1-standard-4'
)

## Tester votre endpoint

In [ ]:

# Make a prediction
response = endpoint.predict(instances=[{"input": ...}])
print(response)